In [31]:
# LOAD PACKAGES 
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
opj = os.path.join
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# For prfpy 
from prfpy_csenf.stimulus import *
from prfpy_csenf.model import Iso2DGaussianModel,Norm_Iso2DGaussianModel
# For dag_prf_utils
from dag_prf_utils.prfpy_functions import Prf1T1M, PrfMulti 
from dag_prf_utils.prfpy_ts_plotter import TSPlotter
from dag_prf_utils.plot_functions import * 
# SCOTOMA PROJECT 
from scot_habrok.load_saved_info import *

# Fix seed (just in case doing anything random)
np.random.seed(0)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
os.getcwd()

'/home4/p307263/programs/scot_habrok/scot_habrok/notebooks'

In [32]:
# Save the notebook outputs 
# SAVE NOTEBOOK + FIGURES
from figure_finder.figure_saver import FigureSaver
from figure_finder.utils import save_running_code

nb_output = '/scratch/p307263/pilot1/derivatives/notebook_timeline'
if not os.path.exists(nb_output):
    os.makedirs(nb_output)

fig_saver = FigureSaver(
    name='scot_HAB_half-2024-07-24',
    path=nb_output,
    fig_overwrite='ow',   
    save_mode       = True, 
    save_svg        = True,
    save_svg_w_code = False,
    save_png        = True,
    save_pdf        = True,
    folder_ow       = True,
)
# save_running_code(fig_saver.path)
as_cols = {
    'AS0'   : '#000000',
    'AS1'   : '#d95f02',
    'AS2'   : '#7570b3',
}

# Load subjects - sped up by multiprocessing
ecc_bins = np.linspace(0,5,6)
# More IP plotting functions

mod_cols = {
    'gauss' : '#3CA969',
    'norm' :  '#8557A7',
}

FOLDER ALREADY EXISTS!
Deleting and remaking folder


In [33]:
task_list = ['AS0', 'AS1', 'AS2']
model_list = ['gauss' ] # , 'norm']
sub_list = ['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07'] 
look_in = '/scratch/p307263/pilot1/derivatives/prf_NM_hrf4pt6_BL_full'
roi_fit = 'all'
mod_cols = {
    'gauss' : '#3CA969',
    'norm' :  '#8557A7',
}
roi_list = ['all', 'v1custom', 'v2custom', 'v3custom', 'v3abcustom', 'v4custom', 'LOcustom', 'TOcustom', 'IPScustom']
# SIMPLIFIED VERSION:
upsampling_factor = 1.7**2 / 0.7**2
print(f'Upsampling factor: {upsampling_factor}')



Upsampling factor: 5.8979591836734695


In [34]:
scotoma_info = get_scotoma_info()
dm_npy = get_design_matrix_npy(['AS0', 'AS1', 'AS2'])
fit_settings = load_yml_settings(hrf_version='new', sub=None)
prfpy_stim = {}
prfpy_mod = {'gauss':{}, 'norm':{}}
cut_vols = 5
for stim in dm_npy.keys():
    prfpy_stim[stim] = PRFStimulus2D(
        screen_size_cm=fit_settings['screen_size_cm'],
        screen_distance_cm=fit_settings['screen_distance_cm'],
        design_matrix=dm_npy[stim][:,:,cut_vols:], 
        axis=0,
        TR=fit_settings['TR']
        )
    prfpy_mod['gauss'][stim] = Iso2DGaussianModel(prfpy_stim[stim])
    prfpy_mod['norm'][stim] = Norm_Iso2DGaussianModel(prfpy_stim[stim])



In [35]:
def simplify_prf_names(prf_name):
    prf_out = prf_name.replace('gauss', 'G')
    prf_out = prf_out.replace('norm', 'N')
    prf_out = prf_out.replace('_run-', 'r')
    if 'r' not in prf_out:
        prf_out = prf_out.replace('_', 'f_') # for full
    return prf_out


# Load the relevant stuff

In [36]:
fit_version = 'full'
prf_obj = {}
for sub in sub_list:
    real_tc = load_data_tc(
        sub=sub, 
        task_list=task_list, 
        look_in=look_in,
        )        
    try:
        prf_pars = load_data_prf(
            sub=sub,
            task_list=task_list,
            look_in=look_in, 
            model_list=model_list,
            roi_fit=roi_fit,
            include=['optimal'], 
            )
    except:
        continue
    prf_obj_list = []
    prf_obj_id = []
    for iT,task in enumerate(task_list):    
        dm_task = task.split('_')[0]
        for model in model_list:
            prf_obj_list.append(TSPlotter(
                prf_params = prf_pars[task][model],
                model=model,
                prfpy_model=prfpy_mod[model][dm_task],))
            prf_obj_id.append(f'{model[0]}_{dm_task}')
    prf_obj[sub] = PrfMulti(prf_obj_list, prf_obj_id)


/scratch/p307263/pilot1/derivatives/prf_NM_hrf4pt6_BL_full/sub-01/ses-1/sub-01_model-gauss_roi-all_hrf-optimal_task-AS0-fits_COLLATED_stage-iter_constr-nelder_desc-prf_params.pkl
/scratch/p307263/pilot1/derivatives/prf_NM_hrf4pt6_BL_full/sub-01/ses-1/sub-01_model-gauss_roi-all_hrf-optimal_task-AS1-fits_COLLATED_stage-iter_constr-nelder_desc-prf_params.pkl
/scratch/p307263/pilot1/derivatives/prf_NM_hrf4pt6_BL_full/sub-01/ses-1/sub-01_model-gauss_roi-all_hrf-optimal_task-AS2-fits_COLLATED_stage-iter_constr-nelder_desc-prf_params.pkl
prf_params.shape[-1]=8
include hrf = True
include rsq = True
prf_params.shape[-1]=8
include hrf = True
include rsq = True
prf_params.shape[-1]=8
include hrf = True
include rsq = True
/scratch/p307263/pilot1/derivatives/prf_NM_hrf4pt6_BL_full/sub-02/ses-1/sub-02_model-gauss_roi-all_hrf-optimal_task-AS0-fits_COLLATED_stage-iter_constr-nelder_desc-prf_params.pkl
/scratch/p307263/pilot1/derivatives/prf_NM_hrf4pt6_BL_full/sub-02/ses-1/sub-02_model-gauss_roi-all_hr

In [37]:
def add2scot(prf_multi):
    # Add distance to scotoma
    for prf_id in prf_multi.id_list:
        for task in ['AS1', 'AS2']:
            this_x = prf_multi.prf_obj[prf_id].pd_params['x'].copy()
            this_y = prf_multi.prf_obj[prf_id].pd_params['y'].copy()
            scot_coord = scotoma_info[task]['scotoma_centre']
            d2s = np.sqrt((this_x-scot_coord[0])**2 + (this_y-scot_coord[1])**2)
            prf_multi.prf_obj[prf_id].pd_params[f'd2s_{task}'] = d2s.copy()
            prf_multi.pd_params[f'{prf_id}-d2s_{task}'] = d2s.copy()
            # Also in terms of prf size 
            this_sigma = prf_multi.prf_obj[prf_id].pd_params['size_1'].copy()
            d2s_sigma = d2s / this_sigma
            prf_multi.prf_obj[prf_id].pd_params[f'd2s_s1_{task}'] = d2s_sigma.copy()
            prf_multi.pd_params[f'{prf_id}-d2s_s1_{task}'] = d2s.copy()
    return prf_multi

for sub in sub_list:
    try:
        prf_obj[sub] = add2scot(prf_obj[sub])
    except:
        continue


In [43]:
from dag_prf_utils.mesh_dash import MeshDash
fs_dir = '/scratch/p307263/pilot1/derivatives/freesurfer'
fs = MeshDash('sub-01', fs_dir=fs_dir)
data_mask1 = (prf_obj['sub-01'].pd_params['g_AS0-rsq']>0) & (prf_obj['sub-01'].pd_params['g_AS0-ecc']<5)
data_pol1 = 
a = fs.add_plotly_surface(
    data=prf_obj['sub-01'].pd_params['g_AS0-pol'].to_numpy(),
    data_mask=data_mask.to_numpy(),
    surf_name='pol', 
    vmin=-np.pi, vmax=np.pi,
    cmap='marco_pol', 
    
)

no dash..
Using fs dir = /scratch/p307263/pilot1/derivatives/freesurfer
No vx to mask
No vx to mask


252330

In [44]:
# a.show()
a.write_html('blah.html')

In [22]:
!ls

blah.html   MAIN_ANLAYSES.ipynb  test_compare_fitters.ipynb
blah.ipynb  test1.ipynb		 test.ipynb


In [12]:
prf_obj['sub-01'].pd_params

,g_AS0-x,g_AS0-y,g_AS0-size_1,g_AS0-amp_1,g_AS0-bold_baseline,g_AS0-hrf_deriv,g_AS0-hrf_disp,g_AS0-rsq,g_AS0-ecc,g_AS0-pol,...,g_AS0-d2s_AS2,g_AS0-d2s_s1_AS2,g_AS1-d2s_AS1,g_AS1-d2s_s1_AS1,g_AS1-d2s_AS2,g_AS1-d2s_s1_AS2,g_AS2-d2s_AS1,g_AS2-d2s_s1_AS1,g_AS2-d2s_AS2,g_AS2-d2s_s1_AS2
0,4.516657,-7.495574,3.491180,0.007231,0.127749,3.241,0.0,0.494041,8.751218,-1.028485,...,8.751218,8.751218,9.371570,9.371570,8.877421,8.877421,3.511772,3.511772,3.062701,3.062701
1,3.001896,-5.841510,2.914007,0.003614,0.235000,3.241,0.0,0.353181,6.567695,-1.096098,...,6.567695,6.567695,9.765427,9.765427,9.187815,9.187815,3.258714,3.258714,2.705554,2.705554
2,1.809550,-4.437434,2.263864,0.002640,0.275151,3.241,0.0,0.317631,4.792211,-1.183591,...,4.792211,4.792211,9.735451,9.735451,9.138485,9.138485,3.126351,3.126351,2.503723,2.503723
3,5.610217,-8.587596,3.482707,0.019392,0.068818,3.241,0.0,0.538506,10.257746,-0.992110,...,10.257746,10.257746,10.357232,10.357232,9.972838,9.972838,5.220683,5.220683,4.885494,4.885494
4,4.807169,-7.133880,3.476750,0.006154,0.002753,3.241,0.0,0.471482,8.602391,-0.977837,...,8.602391,8.602391,7.892387,7.892387,7.479337,7.479337,3.632006,3.632006,3.281831,3.281831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617219,-2.001133,-1.216521,0.100000,0.311207,-2.690960,3.241,0.0,0.014009,2.341892,-2.595373,...,2.341892,2.341892,4.997620,4.997620,5.303945,5.303945,6.007268,6.007268,5.154183,5.154183
617220,2.018322,-3.022213,0.100000,0.347749,-0.719512,3.241,0.0,0.021787,3.634198,-0.981989,...,3.634198,3.634198,4.915991,4.915991,5.240154,5.240154,9.759212,9.759212,9.992076,9.992076
617221,-0.265241,2.632969,0.100000,0.485740,-3.000000,3.241,0.0,0.011789,2.646295,1.671196,...,2.646295,2.646295,2.815892,2.815892,4.069117,4.069117,6.206380,6.206380,5.731496,5.731496
617222,1.956609,2.605497,0.100000,0.623385,-0.780503,3.241,0.0,0.020870,3.258363,0.926685,...,3.258363,3.258363,2.450194,2.450194,3.012795,3.012795,4.217582,4.217582,3.924232,3.924232
